In [1]:
import llama_cpp
import instructor

#from llama_cpp.llama_speculative import LlamaPromptLookupDecoding
from pydantic import BaseModel, Field


llama = llama_cpp.Llama(
    model_path="../models/Hermes-2-Pro-Llama-3-8B-Q4.gguf",
    n_gpu_layers=-1,
    n_ctx=2048,
    chat_format="llama-3",
    verbose=False,
)


create = instructor.patch(
    create=llama.create_chat_completion_openai_v1,
    mode=instructor.Mode.JSON_SCHEMA, 
)



In [2]:
from enum import Enum
from typing import List, Optional

from parser.model import QuestionClassification


def extract_question_data(
    primary_text: str, sub_text: str | None = None
) -> QuestionClassification:
    # Define a mapping of question types to identifying characteristics
    question_type_hints = {
        "TABLE": ["table", "matrix"],
        "LINKED_LIST_MODIFICATION": ["linked list"],
        "CONVERT_INFIX_TO_POSTFIX_WITH_STACK": ["infix to postfix", "stack", "parentheses"],
        "CONVERT_INFIX_TO_POSTFIX_WITHOUT_STACK": ["infix to postfix", "stack", "parentheses"],
        "BASE_CONVERSION": ["base conversion", "convert", "base"],
        "CODE_FILL_BLANKS": ["fill in the blanks", "complete the code", "fill"],
        "CODE_FREE_RESPONSE": ["write a function", "implement", "develop"],

        # Add more types and their identifying keywords as needed
    }

    # Function to determine the question type based on text
    def determine_question_type(text: str) -> List[str]:
        question_types = []
        for question_type, keywords in question_type_hints.items():
            if any(keyword in text.lower() for keyword in keywords):
                question_types.append(question_type)
        return question_types

    # Determine the question type for the prompt
    question_type = determine_question_type(primary_text)
    if sub_text:
        sub_question_type = determine_question_type(sub_text)
        if len(sub_question_type) > 0:
            question_type = sub_question_type

    # Construct the prompt with a hint about the question type
    prompt: str = (
        f"Extract the expected input type for the following exam question: "
        f"Main Question: <text>{primary_text}</text>"
        f"{' Sub Question: <text>' + sub_text + '</text>' if sub_text is not None else ''}"
        f" This question is may be one of the following types: {question_type}."
    )

    extraction: QuestionClassification = create(
        messages=[
            {
                "role": "system",
                "content": (
                    "You are an expert at analyzing exam questions. "
                    "Your task is to determine the type of input expected for each question. "
                    "Focus on identifying the expected input type based on the question's context. "
                    "Do not attempt to solve the question itself."
                ),
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        response_model=QuestionClassification,
    )
    return extraction


In [3]:
from typing import List
from parser.model import Section
from parser.parse import main

sections: List[Section] = main("../fe_files/exams/FE-Aug23.pdf")


for section in sections:
    for question in section.questions:
        input_question = question.text
        print(input_question)
        if question.sub_questions is not None and len(question.sub_questions) > 0:
            for sub_question in question.sub_questions:
                print(sub_question.text)
                extraction: QuestionClassification = extract_question_data(input_question, sub_question.text)
                sub_question.classification = extraction
        else:
            extraction: QuestionClassification = extract_question_data(input_question)
            question.classification = extraction

class Document(BaseModel):
    sections: List[Section]

document = Document(sections=sections)

# Write pydantic models to JSON file
with open("document_classified.json", "w") as json_file:
    json_file.write(document.model_dump_json())
    

The struct Monster_List maintains a list of monsters using a dynamically sized array of pointers to 
Monster. A function prototype is given for a function initializeMonster, which takes in a pointer to a 
Monster that must already be pointing to memory that is allocated , and then fills that memory with 
information about a default monster. Write a function getDefaultMonsters which takes in a positive integer 
n, creates a pointer to a Monster_List, allocates room for it, and then fills it with n default Monster s, and 
then returns a pointer to the Monster_list created. (Note: You must call initializeMonster in your solution.)  
 
typedef struct Monster  { 
    // Details not necessary to solve the problem.  
} Monster; 
 
typedef struct Monster_List  { 
    Monster** mArray ; 
    int numMonsters;  
} Monster_List ; 
 
// Initializes the monster pointed to by mPtr to be the default  
// monster.  
void initializeMonster(Monster* mPtr);  
 
Monster_List*  getDefaultMonsters (int n) { 